<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://emdros.org" target="_blank"><img align="left" src="files/images/Emdros-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="right" src="images/VU-ETCBC-xsmall.png"/></a>

# Verbs of movement

Try to find the verbs of movement

In [1]:
import sys
import collections
import subprocess

import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.5.0
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: http://shebanq-doc.readthedocs.org/en/latest/texts/welcome.html



In [5]:
API = fabric.load('etcbc4b', 'lexicon', 'movement', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        oid otype monads
        function
        lex g_word_utf8 trailer_utf8 prs uvf sp gloss
        book chapter verse label number
    ''',''),
    "prepare": prepare,
    "primary": False,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: UP TO DATE
  0.10s INFO: USING DATA COMPILED AT: 2015-05-04T13-46-20
  0.10s DETAIL: COMPILING a: UP TO DATE
  0.11s INFO: USING DATA COMPILED AT: 2015-05-04T14-07-34
  0.11s DETAIL: keep main: G.node_anchor_min
  0.12s DETAIL: keep main: G.node_anchor_max
  0.12s DETAIL: keep main: G.node_sort
  0.12s DETAIL: keep main: G.node_sort_inv
  0.13s DETAIL: keep main: G.edges_from
  0.13s DETAIL: keep main: G.edges_to
  0.13s DETAIL: keep main: F.etcbc4_db_monads [node] 
  0.14s DETAIL: keep main: F.etcbc4_db_oid [node] 
  0.14s DETAIL: keep main: F.etcbc4_db_otype [node] 
  0.14s DETAIL: keep main: F.etcbc4_ft_function [node] 
  0.14s DETAIL: keep main: F.etcbc4_ft_g_word_utf8 [node] 
  0.14s DETAIL: keep main: F.etcbc4_ft_lex [node] 
  0.14s DETAIL: keep main: F.etcbc4_ft_number [node] 
  0.15s DETAIL: keep main: F.etcbc4_ft_prs [node] 
  0.15s DETAIL: keep main: F.etcbc4_ft_sp [node] 
  0.15s DETAIL: keep main: F.etcbc4_f

# First approximation

Find all verbs in a predicate of a clause that has a locative constituent. Mark those verbs as location sensitive.

# Second approximation

If there is a marker for direction on any word in the clause with a location sensitive verb, mark the verb as a verb of movement.

Download the result files from my SURFdrive: [tab separated file](https://surfdrive.surf.nl/files/public.php?service=files&t=9c80db85478ae91a74cb3408a24a1b14) and a formatted [openoffice spreadsheet](https://surfdrive.surf.nl/files/public.php?service=files&t=38df9044eb5ac29a09a9daff81e1f276)

In [27]:
nlclauses = 0
lclauses = set()
mclauses = set()
lverbs = {}
mverbs = {}
freq = collections.Counter()
contxt = collections.defaultdict(lambda: [])

for p in F.otype.s('phrase'):
    if F.function.v(p) == 'Pred':
        nlclauses += 1
        c = L.u('clause', p)
        is_move = False
        is_move_x = False
        transitive = False
        for v in L.d('word', c):
            if F.uvf.v(v) == 'H':
                is_move = True
        for pp in L.d('phrase', c):
            if F.function.v(pp) == 'Loca':
                lclauses.add(c)
                if is_move:
                    mclauses.add(c)
                    is_move_x = True
            elif F.function.v(pp) == 'Objc':
                transitive = True
        if is_move_x:
            for v in L.d('word', p):
                if F.sp.v(v) == 'verb':
                    lex = F.lex.v(v)
                    gloss = F.gloss.v(v)
                    lverbs[lex] = gloss
                    if is_move: mverbs[lex] = F.gloss.v(v)
                    freq[lex] += 1
                    bk = F.book.v(L.u('book', v))
                    ch = F.chapter.v(L.u('chapter', v))
                    vs = F.verse.v(L.u('verse', v))
                    txt = ''.join('{}{}'.format(F.g_word_utf8.v(w), F.trailer_utf8.v(w)) for w in L.d('word', c))
                    contxt[lex].append((txt, 'T' if transitive else 'I', lex, gloss, vs, ch, bk))
                    
print('{} verbs of motion of {} verbs of location of {} clauses with a predicate'.format(
    len(mverbs), len(lverbs), nlclauses))

61 verbs of motion of 61 verbs of location of 57058 clauses with a predicate


In [28]:
of = outfile('lverbs.csv')
for v in sorted(lverbs, key=lambda x: (-freq[x], x)):
    of.write('{},{},{}\n'.format(v, lverbs[v], freq[v]))
of.close()
of = outfile('mverbs.csv')
for v in sorted(mverbs, key=lambda x: (-freq[x], x)):
    of.write('{},{},{}\n'.format(v, mverbs[v], freq[v]))
of.close()
of = outfile('context.csv')
for v in sorted(contxt, key=lambda x: (-freq[x], x)):
    for occ in contxt[v]:
        of.write('{}\n'.format('{},{},{},{},{},{},{}'.format(*occ).replace('\n', ' ')))
of.close()

In [6]:
!head -n 20 {my_file('mverbs.csv')}

HJH[,be,140
<FH[,make,126
NTN[,give,73
BNH[,build,68
R>H[,see,66
MWT[,die,58
BW>[,come,48
DBR[,speak,45
>KL[,eat,44
CM<[,hear,43
NKH[,strike,36
QR>[,call,31
FJM[,put,28
<LH[,ascend,25
JCB[,sit,25
QBR[,bury,25
MLK[,be king,24
XZQ[,be strong,23
<MD[,stand,20
CWB[,return,20
